In [172]:
import requests # requests機能をインポートします。
import pandas as pd
REQUEST_URL = 'https://suumo.jp/chintai/tokyo/sc_nakano/?page={}' #アクセス先をREQUEST_URLを代入します。
target_URL=REQUEST_URL.format(1)
res = requests.get(target_URL) #リクエストしたデータをresに代入します。
soup = BeautifulSoup(res.text)
res.encoding = 'utf-8' # resの文字データがISO-8859-1なので、utf-8に変換して文字化けを防止します。

In [151]:
from bs4 import BeautifulSoup # BeautifulSoupの機能をインポートします。
soup = BeautifulSoup(res.text,"html.parser") # BeautifulSoup(解析したいデータ,解析する方法)を指定し、解析したデータをsoupに代入します。

In [152]:
contents = soup.find_all('div',class_='cassetteitem')

In [153]:
len(contents)

20

In [154]:
content = contents[0]

In [155]:
detail = content.find('div',class_='cassetteitem-detail') #物件・建物情報の変数に格納
table = content.find('table',class_='cassetteitem_other') #各部屋の情報の変数に格納

In [156]:
#物件・建物情報の抽出
title = detail.find('div',class_='cassetteitem_content-title').text
address = detail.find('li',class_='cassetteitem_detail-col1').text
access = detail.find('li',class_='cassetteitem_detail-col2').text
age = detail.find('li',class_='cassetteitem_detail-col3').text

In [157]:
title,address,access,age

('西武新宿線 鷺ノ宮駅 2階建 築15年',
 '東京都中野区白鷺２',
 '\n西武新宿線/鷺ノ宮駅 歩8分\n西武新宿線/下井草駅 歩14分\nＪＲ中央線/阿佐ケ谷駅 歩25分\n',
 '\n築15年\n2階建\n')

In [158]:
#部屋情報の取得
tr_tags = table.find_all('tr',class_='js-cassette_link')
tr_tag = tr_tags[0]

In [159]:
floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]

In [160]:
floor,price,first_fee,capacity

(<td>
 											1階</td>,
 <td>
 <ul>
 <li><span class="cassetteitem_price cassetteitem_price--rent"><span class="cassetteitem_other-emphasis ui-text--bold">9.1万円</span></span></li>
 <li><span class="cassetteitem_price cassetteitem_price--administration">2000円</span></li>
 </ul>
 </td>,
 <td>
 <ul>
 <li><span class="cassetteitem_price cassetteitem_price--deposit">9.1万円</span></li>
 <li><span class="cassetteitem_price cassetteitem_price--gratuity">-</span></li>
 </ul>
 </td>,
 <td>
 <ul>
 <li><span class="cassetteitem_madori">1LDK</span></li>
 <li><span class="cassetteitem_menseki">37.26m<sup>2</sup></span></li>
 </ul>
 </td>)

In [161]:
#賃料と管理費を格納
fee, management_fee = price.find_all("li")
#敷金と礼金を格納
deposit, gratuity = first_fee.find_all("li")
#間取りと面積を格納
madori, menseki = capacity.find_all("li")

In [162]:
print(fee)
print(management_fee)
print()

print(deposit)
print(gratuity)
print()

print(madori)
print(menseki)
print()

<li><span class="cassetteitem_price cassetteitem_price--rent"><span class="cassetteitem_other-emphasis ui-text--bold">9.1万円</span></span></li>
<li><span class="cassetteitem_price cassetteitem_price--administration">2000円</span></li>

<li><span class="cassetteitem_price cassetteitem_price--deposit">9.1万円</span></li>
<li><span class="cassetteitem_price cassetteitem_price--gratuity">-</span></li>

<li><span class="cassetteitem_madori">1LDK</span></li>
<li><span class="cassetteitem_menseki">37.26m<sup>2</sup></span></li>



In [163]:
# 取得した情報を辞書に格納
d = {
    'title':title,
    'address':address,
    'access':access,
    'age':age,
    'floor':floor.text,
    'fee':fee.text,
    'management_fee':management_fee.text,
    'deposit':deposit.text,
    'gratuity':gratuity.text,
    'madori':madori.text,
    'menseki':menseki.text
}

In [164]:
d

{'title': '西武新宿線 鷺ノ宮駅 2階建 築15年',
 'address': '東京都中野区白鷺２',
 'access': '\n西武新宿線/鷺ノ宮駅 歩8分\n西武新宿線/下井草駅 歩14分\nＪＲ中央線/阿佐ケ谷駅 歩25分\n',
 'age': '\n築15年\n2階建\n',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t1階',
 'fee': '9.1万円',
 'management_fee': '2000円',
 'deposit': '9.1万円',
 'gratuity': '-',
 'madori': '1LDK',
 'menseki': '37.26m2'}

In [168]:
d_list=[]

contents = soup.find_all('div',class_='cassetteitem')

for content in contents:
    detail = content.find('div',class_='cassetteitem-detail') #物件・建物情報の変数に格納
    table = content.find('table',class_='cassetteitem_other') #各部屋の情報の変数に格納
    
    #物件・建物情報の抽出
    title = detail.find('div',class_='cassetteitem_content-title').text
    address = detail.find('li',class_='cassetteitem_detail-col1').text
    access = detail.find('li',class_='cassetteitem_detail-col2').text
    age = detail.find('li',class_='cassetteitem_detail-col3').text
    
    #各部屋の情報
    tr_tags = table.find_all('tr',class_='js-cassette_link')
    for tr_tag in tr_tags:
        floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]
        fee, management_fee = price.find_all("li")
        deposit, gratuity = first_fee.find_all("li")
        madori, menseki = capacity.find_all("li")
        
        d = {
            'title':title,
            'address':address,
            'access':access,
            'age':age,
            'floor':floor.text,
            'fee':fee.text,
            'management_fee':management_fee.text,
            'deposit':deposit.text,
            'gratuity':gratuity.text,
            'madori':madori.text,
            'menseki':menseki.text
        }
        
        d_list.append(d)
        
        

In [169]:
from pprint import pprint

In [171]:
pprint(d_list[1])

{'access': '\n都営大江戸線/新江古田駅 歩8分\n西武池袋線/東長崎駅 歩11分\n都営大江戸線/落合南長崎駅 歩13分\n',
 'address': '東京都中野区江原町１',
 'age': '\n築8年\n3階建\n',
 'deposit': '10.2万円',
 'fee': '10.2万円',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t1階',
 'gratuity': '10.2万円',
 'madori': '1LDK',
 'management_fee': '-',
 'menseki': '35.5m2',
 'title': 'ＥＨＡＲＡ\u3000ＧＡＲＤＥＮ'}


In [173]:
d_list[:2]

[{'title': '西武新宿線 鷺ノ宮駅 2階建 築15年',
  'address': '東京都中野区白鷺２',
  'access': '\n西武新宿線/鷺ノ宮駅 歩8分\n西武新宿線/下井草駅 歩14分\nＪＲ中央線/阿佐ケ谷駅 歩25分\n',
  'age': '\n築15年\n2階建\n',
  'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t1階',
  'fee': '9.1万円',
  'management_fee': '2000円',
  'deposit': '9.1万円',
  'gratuity': '-',
  'madori': '1LDK',
  'menseki': '37.26m2'},
 {'title': 'ＥＨＡＲＡ\u3000ＧＡＲＤＥＮ',
  'address': '東京都中野区江原町１',
  'access': '\n都営大江戸線/新江古田駅 歩8分\n西武池袋線/東長崎駅 歩11分\n都営大江戸線/落合南長崎駅 歩13分\n',
  'age': '\n築8年\n3階建\n',
  'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t1階',
  'fee': '10.2万円',
  'management_fee': '-',
  'deposit': '10.2万円',
  'gratuity': '10.2万円',
  'madori': '1LDK',
  'menseki': '35.5m2'}]

In [175]:
df = pd.DataFrame(d_list)

In [178]:
df.head()


,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,西武新宿線 鷺ノ宮駅 2階建 築15年,東京都中野区白鷺２,\n西武新宿線/鷺ノ宮駅 歩8分\n西武新宿線/下井草駅 歩14分\nＪＲ中央線/阿佐ケ谷駅...,\n築15年\n2階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,9.1万円,2000円,9.1万円,-,1LDK,37.26m2
1,ＥＨＡＲＡ ＧＡＲＤＥＮ,東京都中野区江原町１,\n都営大江戸線/新江古田駅 歩8分\n西武池袋線/東長崎駅 歩11分\n都営大江戸線/落合...,\n築8年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,10.2万円,-,10.2万円,10.2万円,1LDK,35.5m2
2,ＥＨＡＲＡ ＧＡＲＤＥＮ,東京都中野区江原町１,\n都営大江戸線/新江古田駅 歩8分\n西武池袋線/東長崎駅 歩11分\n都営大江戸線/落合...,\n築8年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,11万円,-,11万円,11万円,1LDK,40.09m2
3,メゾンドオピュール,東京都中野区弥生町５,\n東京メトロ丸ノ内線/中野富士見町駅 歩5分\n東京メトロ丸ノ内線/方南町駅 歩13分\n...,\n築8年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t3階,11.2万円,3000円,-,11.2万円,1LDK,49.62m2
4,メゾンドオピュール,東京都中野区弥生町５,\n東京メトロ丸ノ内線/中野富士見町駅 歩5分\n東京メトロ丸ノ内線/方南町駅 歩13分\n...,\n築8年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t3階,11.2万円,3300円,-,11.2万円,1LDK,49.62m2


In [179]:
df.shape

(50, 11)

In [180]:
len(df.title.unique())

20

In [181]:
# CSV出力
df.to_csv('SUUMO_nakano.csv',index=None)
